In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [1]:
!pwd


/home/idhurwitz/mlops-zoomcamp/01-intro/homework-2023


In [2]:
# download the january and february 2022. yellow taxi trip records
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
    

--2023-05-15 21:00:18--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.141.158, 13.225.141.174, 13.225.141.98, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.141.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  76.0MB/s    in 0.5s    

2023-05-15 21:00:19 (76.0 MB/s) - ‘./data/yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]



In [3]:
!wget -P ./data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-15 21:01:20--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.97, 52.85.39.117, 52.85.39.65, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2022-02.parquet’

yellow_tripdata_202 100%[===================>]  43.50M   123MB/s    in 0.4s    

2023-05-15 21:01:21 (123 MB/s) - ‘./data/yellow_tripdata_2022-02.parquet’ saved [45616512/45616512]



In [4]:
df_jan = pd.read_parquet("./data/yellow_tripdata_2022-01.parquet") 
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
df_jan.shape

(2463931, 19)

there are 19 columns in in the january file

### Question 2

In [6]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']

In [7]:
df_jan['duration'] = df_jan.duration.dt.total_seconds() / 60

In [8]:
df_jan.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

closest answer for the january duration is 46.45

### Question 3

In [9]:
old_len = len(df_jan)

In [10]:
df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()

In [11]:
df_jan.shape

(2421440, 20)

In [12]:
len(df_jan) / old_len

0.9827547930522406

there are 98 percent of the records left after dropping the extreme outlieres

### Question 4

In [13]:
categorical = ['PULocationID', 'DOLocationID']

In [14]:
# the pick up and dropoff location ids are in int but we want them as strings so they can be one-hot encoded
df_jan[categorical] = df_jan[categorical].astype('str')

In [15]:
train_dicts = df_jan[categorical].to_dict(orient='records')

In [16]:
dv = DictVectorizer()

In [17]:
X_train = dv.fit_transform(train_dicts)

In [18]:
X_train.shape

(2421440, 515)

The dimensionality of the the feature vector is 515

### Question 5

In [19]:
y_train = df_jan['duration'].values 

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [21]:
y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

6.986190834966621

closest answer for the rmse on training set is 6.99

### Question 6

In [35]:
def prepare_df(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df.duration.dt.total_seconds() / 60
    
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1).astype(int).astype('str')
    
    return df

In [26]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [36]:
#df_val = read_data("./data/yellow_tripdata_2022-02.parquet")
df_val = prepare_df("./data/yellow_tripdata_2022-02.parquet")

In [37]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [38]:
X_val = dv.transform(val_dicts)

In [39]:
y_pred = lr.predict(X_val)
y_val = df_val['duration'].values
rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

7.786408960592508

closest answer is 7.79